In [0]:
%tensorflow_version 2.x
%load_ext tensorboard

import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
from tensorflow import keras

# We are running on Tensorflow 2.x
print(tf.__version__)

# Project Overview

## Stanford Dog Dataset

This simple Dog-breed classifier is based on [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/)

### Retrieving through TFDS

In [0]:
import tensorflow_datasets as tfds
(raw_train_dataset, raw_validation_dataset, raw_test_dataset), dataset_info = tfds.load('stanford_dogs', 
                                  with_info=True, 
                                  split=['train[:70%]', 'train[70%:90%]', 'train[90%:]'],
                                  as_supervised=True,
                                )

breed_names = dataset_info.features['label'].names

## Preprocessing the Dataset

In [0]:
IMG_SIZE = 224 # All images will be resized to 224x224 (which is ideal for MobileNetV2)
BATCH_SIZE= 20

In [0]:
def format_image(image):
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image

# Standard preprocess dataset
def preprocess_dataset(dataset):
  dataset = dataset.map(lambda image, label: (format_image(image), label))
  dataset = dataset.shuffle(1000).batch(BATCH_SIZE)
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset


In [0]:
# Standard train dataset, without any augmentation
std_train_dataset = raw_train_dataset.map(lambda image, label: (format_image(image), label))

validation_dataset = preprocess_dataset(raw_validation_dataset)

## Image Augmentation


In [0]:
# Central crop, similar like zooming
def central_crop(image):
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.central_crop(image, 0.7)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image

# Horizontal flip
def horizontal_flip(image):
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.flip_left_right(image)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image

# Rotate 90degree
def rot90(image):
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.rot90(image)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image

# Rotate 270 degree, which is the other way
def rot270(image):
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.rot90(image, k=3)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image

cropped_dataset = raw_train_dataset.shuffle(1000).take(8000).map(lambda image, label: (central_crop(image), label))
horflip_dataset = raw_train_dataset.shuffle(1000).take(1000).map(lambda image, label: (horizontal_flip(image), label))
rot90_dataset = raw_train_dataset.shuffle(1000).take(3500).map(lambda image, label: (rot90(image), label))
rot270_dataset = raw_train_dataset.shuffle(1000).take(3500).map(lambda image, label: (rot270(image), label))

# Combine all dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE
augmented_dataset = std_train_dataset.concatenate(cropped_dataset)
augmented_dataset = augmented_dataset.concatenate(horflip_dataset)
augmented_dataset = augmented_dataset.concatenate(rot90_dataset)
augmented_dataset = augmented_dataset.concatenate(rot270_dataset)

# Shuffle, repeat, batch, prefetch
augmented_dataset = augmented_dataset.shuffle(1000).repeat().batch(BATCH_SIZE).prefetch(AUTOTUNE)

## Creating Neural Network

We are using MobileNetV2 as our base model.

In [0]:
def get_model():
  model = keras.Sequential()

  mobilev2 = keras.applications.MobileNetV2(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

  # Fine-tuning the base model
  mobilev2.trainable = True
  for layer in mobilev2.layers:
    if layer.name.find('Conv_1'):
      layer.trainable = True
    elif layer.name == 'out_relu':
      layer.trainable = True
    else:
      layer.trainable = False

  # Using MobileNetV2
  model.add(mobilev2)

  # Flatten the convolution and put to DNN
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(1024, activation='relu'))
  model.add(keras.layers.Dropout(0.3))
  model.add(keras.layers.Dense(512, activation='relu'))
  
  # Lastly use DNN with output shape 120 because there are 120 breeds
  model.add(keras.layers.Dense(120, activation='softmax'))

  return model

In [0]:
model = get_model()
model.summary()

## Training our Neural Network

In [0]:
# Remove logs of previous training to use Tensorboard
!rm -rf ./logs/ 

In [0]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [0]:
model.fit(augmented_dataset,
          validation_data=validation_dataset,
          epochs=50,
          steps_per_epoch=200, 
          validation_steps=50,
          callbacks=[tensorboard_callback])

### Plotting the result

In [0]:
import matplotlib.pyplot as plt

def plot_result(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(1, len(acc) + 1)
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training & Validation accuracy') 
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training loss') 
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training & Validation loss')
  plt.legend()

  plt.show()

plot_result(model.history)

### Viewing in Tensorboard

In [0]:
%tensorboard --logdir logs/fit

## Evaluating our Neural Network
Now that the training phase has ended, let's evaluate with our test dataset. Before that, we should preprocess our test_dataset


In [0]:
test_dataset = preprocess_dataset(raw_test_dataset)
model.evaluate(test_dataset)

## Evaluating with file upload

In [0]:
# Standard function to decode and resize image
def decode_img_from_path(file_path):
  image = tf.io.read_file(file_path)
  image = tf.image.decode_jpeg(image, channels=3) # The file should be JPEG
  # image = tf.image.decode_png(image, channels=3) # If the file is PNG
  return format_image(image) # Also used in Validation and Test Dataset

In [0]:
from google.colab import files

uploaded = files.upload()
upload_dataset = tf.data.Dataset.from_tensor_slices([list(uploaded.keys())])

### Upload manually with Google Colab toolbar

In case our file is too big for `files.upload()`.

In [0]:
upload_dataset = tf.data.Dataset.from_tensor_slices(['some_dog_image.jpg'])

### Predicting the uploaded files

In [0]:
upload_dataset = upload_dataset.map(lambda image_path: decode_img_from_path(image_path))
upload_dataset = upload_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

predictions = model.predict(upload_dataset)

for pred in predictions:
  breeds_pct, breeds_idx = tf.math.top_k(pred, k=3)
  breeds_pct = breeds_pct.numpy()
  breeds_idx = breeds_idx.numpy()
  print("Predictions:")
  for i in range(3):
    print("{} ({}%)".format(breed_names[breeds_idx[i]], breeds_pct[i]))

## Saving our Model

Since I'm satisfied with the performance of the model, I want to export the model to be able to be used by my Web app.

HDF5 file will contain:

- Architecture of model to recreate
- Weights of the model
- Training configuration
- State of the optimizer

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Save it in Google Drive
model.save('/content/drive/My Drive/ML Project/Dog Breeds/stanford_dogs_20200307.h5') # Use .h5 extension to save to HDF5

### Loading our model

In [0]:
loaded_model = keras.models.load_model('/content/drive/My Drive/ML Project/Dog Breeds/stanford_dogs_20200307.h5')
loaded_model.summary()

loaded_model.evaluate(test_dataset)

## References

Dataset
- [Stanford Dogs dataset for Fine-Grained Visual Categorization](http://vision.stanford.edu/aditya86/ImageNetDogs/)
- [Tensorflow Datasets](https://www.tensorflow.org/datasets)

Preprocessing Data
- [tf.data: Build TensorFlow input pipelines](https://www.tensorflow.org/guide/data)
- [Load images using tf.data](https://www.tensorflow.org/tutorials/load_data/images#load_using_tfdata)

MobileNetV2
- [MobileNetV2: The Next Generation of On-Device Computer Vision Networks](https://ai.googleblog.com/2018/04/mobilenetv2-next-generation-of-on.html)

Saving Keras Model
- [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load)
- [FAQ - Keras Documentation](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model)